层和块

1. 定义一个MLP块（包含256个单元的隐藏层和一个10维输出层）

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [8]:
## 随机生成一个2x10的张量
X = torch.randn(2, 20)
X

tensor([[-3.9554e-01, -6.8164e-01, -5.4824e-01, -1.1032e+00, -3.2836e-02,
         -9.5626e-01, -2.5210e-01, -5.4677e-01, -7.6672e-01, -1.8748e+00,
          1.1100e-01, -8.3585e-03,  6.1015e-02,  4.7751e-01, -1.7325e+00,
         -2.5108e-01, -2.8593e-01, -3.9150e-01, -3.9549e-01, -2.6752e-01],
        [ 5.5885e-01,  1.4754e-03,  7.2378e-01, -2.3723e-01, -1.5148e-01,
          1.3982e+00, -1.3631e+00,  7.8454e-02, -5.1196e-01,  1.1061e+00,
          6.6938e-01, -6.7595e-02, -4.0797e-01,  2.6570e-01, -1.1499e+00,
         -3.8921e-01,  4.2028e-01,  1.5264e+00, -2.5360e-01,  6.5580e-03]])

In [5]:
## 从头编写一个MLP类（Module子类）
class MLP(nn.Module):
    ## 初始化
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    ## 定义前向传播函数
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [9]:
## 创建MLP实例
net = MLP()
net(X)

tensor([[-0.3837, -0.1129, -0.0970, -0.1217,  0.1737, -0.2268,  0.0224,  0.2441,
         -0.1296, -0.0910],
        [-0.2863, -0.1819, -0.1009, -0.0800, -0.1360,  0.0827, -0.1208,  0.0706,
         -0.0573,  0.0006]], grad_fn=<AddmmBackward0>)

In [11]:
print(net.parameters)

<bound method Module.parameters of MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)>


2. 定义一个顺序块

In [ ]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [13]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2064,  0.2374,  0.0619,  0.0460, -0.1093, -0.1236, -0.0008, -0.2300,
         -0.0098, -0.2635],
        [-0.0696,  0.2376,  0.3117, -0.2089, -0.1829, -0.1356, -0.0555, -0.0578,
          0.1043,  0.1046]], grad_fn=<AddmmBackward0>)

In [14]:
## 同样也可以使用Sequential类
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2248, -0.2591,  0.0273,  0.1665,  0.2015,  0.3734, -0.1806,  0.1012,
          0.1398,  0.2632],
        [-0.0628, -0.2370,  0.0587, -0.1741,  0.0645,  0.1949, -0.1770, -0.1823,
         -0.3528, -0.0610]], grad_fn=<AddmmBackward0>)

3. 个性化设置块

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        ## 随机生成权重值，且不设置梯度，即不更新权重参数
        self.rand_weight = torch.rand((20, 20), requires_grad = False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self , X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        ## 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [17]:
net = FixedHiddenMLP()
net(X)

tensor(0.0209, grad_fn=<SumBackward0>)